Once the prediction of the mask is done by the U-Net, that prediction needs to be process to remove all unwanted areas. Once that's done, the resulting mask should be apply to the xRay in order to keep only the hand and feed the prediction NN.

In [34]:
import matplotlib.pyplot as plt 
import numpy as np
import cv2
import skimage.io as io
import os

In [35]:
predict_dir = 'data/hands/test/predict/' #output of the U-Net on the testing xRay
images_dir = 'data/hands/test/image/'    #testing xRay
output_dir = 'data/hands/test/output/'   #xRay masked (input of the next NN)
i_fmt = 'png'
o_fmt = 'png'
size = (256,256)

In [36]:
for file in os.listdir(images_dir):
    if not file.endswith(i_fmt): continue
    
    xRay_path  = images_dir  + file
    outfile    = output_dir  + file
    prediction = predict_dir + file
    
    im1 = cv2.imread(prediction) # read the predicted mask
    im1[-10:,:,:] = 255 #paint a white stripe in the bottom to be sure the fram doesn't touch the hand
    im2 = cv2.cvtColor(im1,cv2.COLOR_BGR2GRAY) #gray scale image
    thresh, im_bw = cv2.threshold(im2, 127, 255, cv2.THRESH_BINARY) #im_bw: binary image
    #get the contours
    contours, hierarchy = cv2.findContours(im_bw,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
    #cv2.drawContours(im2, contours,-1, (100,255,0),3)
    #plt.imshow(im2) 
    #create a black picture to draw the contour
    im3 = np.ones([size[0], size[1], 3], dtype=np.uint8)*255
    #get the contour of the hand and discard the rest
    areaArray=[]
    frame_area = size[0] * size[1]
    for i, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if area <= frame_area*0.01 or area > frame_area*0.9:  #discard the frame and small areas
            continue
        areaArray.append(contour)
    #there should be only one contour selected, otherwise start debugging...
    if len(areaArray)>1:
        print("more than one hspae detected as hand")
    handContour = areaArray[0]
    #draw the contour in the black picture
    cv2.drawContours(im3, handContour, -1, (0,0,0), 1)
    
    # Mask used to flood filling.
    # Notice the size needs to be 2 pixels than the image.
    h, w = im3.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    # Floodfill from point (0, 0)
    cv2.floodFill(im3, mask, (0,0), 0);
    #plt.imshow(im3)
    #if I want to save the mask: io.imsave("something.png",im3)

    xray = cv2.imread(xRay_path) #raw RGB image
    idx=(im3==0) #get the indices of the mask that are 0

    xray[idx]=0 #change all those pixels (which are not part of the hand) to 0
    io.imsave(output,xray)